In [1]:
from IPython.display import clear_output
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import re
import dropbox
import sys
import pandas as pd
import numpy as np
import imageio.v2 as imageio
import matplotlib.pyplot as plt
import cv2
import logging
import datetime
import glob
import json
import scipy
import matplotlib as mpl

from subprocess import call
from tifffile import imwrite
from tqdm import tqdm
from pathlib import Path
from amftrack.util.dbx import (
    upload_folder,
    download,
    read_saved_dropbox_state,
    save_dropbox_state,
    load_dbx,
    get_dropbox_folders_prince,
    get_dropbox_video_folders,
    download_video_folders_drop,
    download_analysis_folders_drop,
)
from amftrack.pipeline.launching.run import (
    run_transfer,
)
from amftrack.pipeline.launching.run_super import run_parallel_transfer
from amftrack.pipeline.launching.run_super import run_parallel_flows
from amftrack.pipeline.functions.transport_processing.high_mag_videos.plot_data import (
    plot_summary,
    save_raw_data,
)
from amftrack.pipeline.functions.transport_processing.high_mag_videos.high_mag_analysis import (
    HighmagDataset,
    VideoDataset,
    EdgeDataset,
    index_videos_dropbox_new,
    analysis_run,
)
from amftrack.pipeline.functions.transport_processing.high_mag_videos.kymo_class import (
    KymoVideoAnalysis,
    KymoEdgeAnalysis,
)
from IPython.display import clear_output

%matplotlib widget
%load_ext autoreload
%autoreload 2
logging.basicConfig(stream=sys.stdout, level=logging.debug)
mpl.rcParams["figure.dpi"] = 200

c:\users\coren\documents\phd\code\amftrack\amftrack\util\dbx.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
# videos_folder = "F:\\AMOLF_Data\\videos\\"
# analysis_folder = "F:\\AMOLF_Data\\analysis\\"

# videos_folder = "/gpfs/scratch1/shared/amftrackflow/videos/"
# analysis_folder = "/gpfs/home6/svstaalduine/Analysis/"
videos_folder = r"C:\Users\coren\Documents\PhD\paper\transport"

analysis_folder = r"C:\Users\coren\Documents\PhD\paper\transport\analysis"
dropbox_address = "/DATA/CocoTransport/20230811_Plate441/"


In [3]:
#%%
video_param_frame = index_videos_dropbox_new(
    analysis_folder,
    videos_folder,
    dropbox_address,
    REDO_SCROUNGING=True,
    # date_start=20230801,
    # date_end=20230813,
    plate_names=None,
)
clear_output(wait=False)

In [4]:
list_video = [11,12,34,35,36,37,75,76,79,80]
download_frame = video_param_frame.copy()
download_frame = download_frame[download_frame['video_int'].isin(list_video)]
download_frame

,imaging_day,storage_path,plate_id,root,strain,treatment,crossing_day,video_int,time_(s),mode,...,ypos,zpos,unique_id,folder,tot_path_drop,record_time,days_after_crossing,magnification,analysis_folder,videos_folder
9,20230811,Dropbox\DATA\CocoTransport,20230811_Plate441,Carrot,C2,001P100N100C,20230807,11,10.0,BF,...,-6.129,13.311,20230811_Plate441_011,CocoTransport/20230811_Plate441/011/Img,DATA/CocoTransport/20230811_Plate441/011/Img,17:22:45,4,4.0,NaN,NaN
10,20230811,Dropbox\DATA\CocoTransport,20230811_Plate441,Carrot,C2,001P100N100C,20230807,12,20.0,F,...,-6.129,13.311,20230811_Plate441_012,CocoTransport/20230811_Plate441/012/Img,DATA/CocoTransport/20230811_Plate441/012/Img,17:23:21,4,4.0,NaN,NaN
32,20230811,Dropbox\DATA\CocoTransport,20230811_Plate441,Carrot,C2,001P100N100C,20230807,34,10.0,BF,...,-4.965,11.910,20230811_Plate441_034,CocoTransport/20230811_Plate441/034/Img,DATA/CocoTransport/20230811_Plate441/034/Img,18:01:03,4,50.0,NaN,NaN
33,20230811,Dropbox\DATA\CocoTransport,20230811_Plate441,Carrot,C2,001P100N100C,20230807,35,10.0,F,...,-4.965,11.910,20230811_Plate441_035,CocoTransport/20230811_Plate441/035/Img,DATA/CocoTransport/20230811_Plate441/035/Img,18:01:24,4,50.0,NaN,NaN
34,20230811,Dropbox\DATA\CocoTransport,20230811_Plate441,Carrot,C2,001P100N100C,20230807,36,10.0,BF,...,-5.929,11.936,20230811_Plate441_036,CocoTransport/20230811_Plate441/036/Img,DATA/CocoTransport/20230811_Plate441/036/Img,18:02:40,4,50.0,NaN,NaN
35,20230811,Dropbox\DATA\CocoTransport,20230811_Plate441,Carrot,C2,001P100N100C,20230807,37,10.0,F,...,-5.929,11.936,20230811_Plate441_037,CocoTransport/20230811_Plate441/037/Img,DATA/CocoTransport/20230811_Plate441/037/Img,18:03:24,4,50.0,NaN,NaN
73,20230811,Dropbox\DATA\CocoTransport,20230811_Plate441,Carrot,C2,001P100N100C,20230807,75,10.0,BF,...,-6.159,11.924,20230811_Plate441_075,CocoTransport/20230811_Plate441/075/Img,DATA/CocoTransport/20230811_Plate441/075/Img,18:35:36,4,50.0,NaN,NaN
74,20230811,Dropbox\DATA\CocoTransport,20230811_Plate441,Carrot,C2,001P100N100C,20230807,76,20.0,F,...,-6.159,11.924,20230811_Plate441_076,CocoTransport/20230811_Plate441/076/Img,DATA/CocoTransport/20230811_Plate441/076/Img,18:36:21,4,50.0,NaN,NaN
77,20230811,Dropbox\DATA\CocoTransport,20230811_Plate441,Carrot,C2,001P100N100C,20230807,79,20.0,BF,...,-5.640,11.932,20230811_Plate441_079,CocoTransport/20230811_Plate441/079/Img,DATA/CocoTransport/20230811_Plate441/079/Img,18:40:19,4,50.0,NaN,NaN
78,20230811,Dropbox\DATA\CocoTransport,20230811_Plate441,Carrot,C2,001P100N100C,20230807,80,10.0,F,...,-5.640,11.932,20230811_Plate441_080,CocoTransport/20230811_Plate441/080/Img,DATA/CocoTransport/20230811_Plate441/080/Img,18:40:37,4,50.0,NaN,NaN


In [5]:
download_video_folders_drop(download_frame, videos_folder)
clear_output(wait=False)
print("All videos downloaded!")

All videos downloaded!


In [9]:
folder_filter = dropbox_address[5:]

img_infos = glob.glob(
    f"{analysis_folder}{folder_filter}/**/video_data.json", recursive=True
)
vid_anls_frame = pd.DataFrame()
for address in img_infos:
    add_info = pd.read_json(address, orient="index").T
    vid_anls_frame = pd.concat([vid_anls_frame, add_info], ignore_index=True)

vid_anls_frame = vid_anls_frame.sort_values("unique_id").reset_index(drop=True)
analysis_frame = vid_anls_frame[vid_anls_frame['video_int'].isin(list_video)]

####################################################################################
### Below code will prepare for those videos to be downloaded to videos_folder.  ###
####################################################################################

print(f"Number of videos to be analyzed: {len(analysis_frame)}")

Number of videos to be analyzed: 6


In [11]:
test_video = KymoVideoAnalysis(
    input_frame=analysis_frame.iloc[3], logging=True, samepos_frame=pd.DataFrame()
)

Data input succesful! Starting edge extraction...


ValueError: autodetected range of [nan, nan] is not finite